In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import seaborn as sns
from model.models import Simulator
import service.predictions as predictions
import pickle
import service.files_service as fs
import csv

In [2]:
simulator = Simulator(dt_from='2016-12-01 8:00',dt_to='2016-12-01 20:00',targets=['Close_Ask_EURRUB'],steps=60,
                       shift=3, fit_model=False)

In [3]:
predictions.compute_and_pickle(simulator,dataset_path='data/brent_wti/ask_bid/transformed/EURRUB/merged/merged.csv')

2016-12-16 18:42:08,194 [INFO ]  compute prediction for Close_Ask_EURRUB : {'to': datetime.datetime(2016, 12, 1, 9, 0), 'from': datetime.datetime(2016, 12, 1, 8, 0)}


Close_Ask_EURRUB
cal_time
cal_hour
cal_minute
cal_dayofweek
cal_dayofyear
Close_Ask_LIGHTCMDUSD_reg
Close_Ask_rolling_mean_LIGHTCMDUSD_reg
Close_Ask_ewma_LIGHTCMDUSD_reg
Close_Ask_rolling_mean_EURRUB_reg
Close_Ask_ewma_EURRUB_reg
Close_Ask_BRENTCMDUSD_reg
Close_Ask_rolling_mean_BRENTCMDUSD_reg
Close_Ask_ewma_BRENTCMDUSD_reg


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']
training data did not have the following fields: f4, f5, f6, f7, f8, f9, f10, f11

In [ ]:
predictions.plot_predictions(simulator)